In [1]:
# We are trying to classify using KNN and convolution operation

In [13]:
import numpy as np
import pandas as pd
import torch
import sklearn as sk
import glob
import PIL
import cv2
from sklearn.model_selection import train_test_split
import torchvision
# from torchsummary import summary
import torch.nn as nn
from tqdm import tqdm
import torch.utils.data as data_utils
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

In [119]:
images = glob.glob('../../cats_dogs/train/*.jpg')

In [125]:
# 1 = DOG
# 0 = CAT
labels = [1 if x.split('/')[-1].split('.')[0] == 'dog' else 0 for x in images]

In [128]:
def split_datasets(data, split_size = 0.05):
    train, val = train_test_split(data, test_size = split_size)
    return train, val

In [129]:
dataset = list(zip(images, labels))

In [130]:
train, val = split_datasets(dataset)

In [19]:
incep_model = torchvision.models.Inception3()

/home/paperspace/.local/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = incep_model.to(device)

In [21]:
model.fc = nn.Linear(2048, 10)

In [22]:
def read_images(img_list, img_shape):
    img_holder = np.zeros((len(img_list), img_shape[0], img_shape[1], 3))
    for x in tqdm(range(len(img_list))):
        img = PIL.Image.open(img_list[x])
        img = img.resize((img_shape[0], img_shape[1]))
        img_holder[x] = img
    img_holder = torch.from_numpy(img_holder)
    img_holder = img_holder.reshape((len(img_list), 3, img_shape[0], img_shape[1]))
    return img_holder

In [23]:
arr_img = read_images([x[0] for x in train], (299, 299))

100%|██████████| 23750/23750 [01:31<00:00, 258.54it/s]


In [24]:
incep_model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [25]:
incep_model = incep_model.to(device)

In [31]:
temp = arr_img[0].unsqueeze(0).float().to(device)

In [32]:
incep_model(temp)

tensor([[ 1.0724e+13, -3.1372e+12,  5.5110e+11,  7.5910e+12,  1.4739e+13,
         -1.6608e+13, -2.4488e+12,  5.7801e+12,  2.9633e+12, -4.0037e+12]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [34]:
dataset = torch.utils.data.TensorDataset(arr_img)
dataset = torch.utils.data.DataLoader(dataset, batch_size=4)

In [50]:
preds_holder = np.zeros((int(len(arr_img)/4), 4, 10))

In [51]:
count = 0
for feature in dataset:
    feature = feature[0].float()
    feature = feature.to(device)
    pred = incep_model(feature)
    preds_holder[count] = pred.detach().cpu()
    count += 1

IndexError: index 5937 is out of bounds for axis 0 with size 5937

In [62]:
updated_preds = preds_holder.reshape(-1, 10)

In [63]:
classifier = KNeighborsClassifier(n_neighbors=2)

In [131]:
updated_labels = np.array(labels).reshape(-1, 1)

In [133]:
classifier.fit(preds_holder.reshape(-1, 10), updated_labels[:23748])

/home/paperspace/.local/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=2)

In [83]:
arr_img_val = read_images([x[0] for x in val], (299, 299))

100%|██████████| 1250/1250 [00:04<00:00, 274.80it/s]


In [84]:
dataset_val = torch.utils.data.TensorDataset(arr_img_val)
dataset_val = torch.utils.data.DataLoader(dataset_val, batch_size=4)

In [134]:
val_labels = [x[1] for x in val]

In [135]:
updated_labels_val = np.array(val_labels).reshape(-1, 1)

In [85]:
preds_holder_val = np.zeros((int(len(val)/4), 4, 10))

In [86]:
count = 0
for feature in dataset_val:
    feature = feature[0].float()
    feature = feature.to(device)
    pred = incep_model(feature)
    preds_holder_val[count] = pred.detach().cpu()
    count += 1

IndexError: index 312 is out of bounds for axis 0 with size 312

In [94]:
updated_preds_val = preds_holder_val.reshape(-1, 10)

In [98]:
val_preds = classifier.predict(preds_holder_val.reshape(-1, 10))

In [100]:
for x in val_preds:
    if x != 0:
        print(x)

In [136]:
classifier.score(preds_holder.reshape(-1, 10), updated_labels[:23748])

0.7518527876031665